In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
---> set the Role
USE ROLE CRYPTO;

---> set the Warehouse
USE WAREHOUSE CRYPTO_DATA;

---> set the Database
USE DATABASE CRYPTOCURRENCY;


In [ ]:
-- Create streams to track changes for incremental processing
CREATE OR REPLACE STREAM RAW_CRYPTO.BTC_RAW_STREAM ON TABLE RAW_CRYPTO.BTC_RAW_DAILY;
CREATE OR REPLACE STREAM RAW_CRYPTO.ETH_RAW_STREAM ON TABLE RAW_CRYPTO.ETH_RAW_DAILY;
CREATE OR REPLACE STREAM RAW_CRYPTO.DOGE_RAW_STREAM ON TABLE RAW_CRYPTO.DOGE_RAW_DAILY;

In [ ]:
SHOW STREAMS IN SCHEMA CRYPTOCURRENCY.RAW_CRYPTO;  --test

SELECT * FROM RAW_CRYPTO.BTC_RAW_DAILY LIMIT 5;  --test

SELECT * FROM RAW_CRYPTO.ETH_RAW_DAILY LIMIT 5;   --test

SELECT * FROM RAW_CRYPTO.DOGE_RAW_DAILY LIMIT 5;   --test

In [ ]:
INSERT INTO HARMONIZED_CRYPTO.CRYPTO_HARMONIZED (
    crypto_symbol, date, open, high, low, close, volume, adjclose, daily_return
)
SELECT 
    'BTC' AS crypto_symbol,
    r.date,
    r.open,
    r.high, 
    r.low,
    r.close,
    r.volume,
    r.adjclose,
    d.daily_return
FROM RAW_CRYPTO.BTC_RAW_DAILY r
LEFT JOIN RAW_CRYPTO.BTC_DAILY_RETURNS d ON r.date = d.date
WHERE r.date >= '2020-01-01';

In [ ]:
INSERT INTO HARMONIZED_CRYPTO.CRYPTO_HARMONIZED (
    crypto_symbol, date, open, high, low, close, volume, adjclose, daily_return
)
SELECT 
    'ETH' AS crypto_symbol,
    r.date,
    r.open,
    r.high, 
    r.low,
    r.close,
    r.volume,
    r.adjclose,
    d.daily_return
FROM RAW_CRYPTO.ETH_RAW_DAILY r
LEFT JOIN RAW_CRYPTO.ETH_DAILY_RETURNS d ON r.date = d.date
WHERE r.date >= '2020-01-01';


In [ ]:
INSERT INTO HARMONIZED_CRYPTO.CRYPTO_HARMONIZED (
    crypto_symbol, date, open, high, low, close, volume, adjclose, daily_return
)
SELECT 
    'DOGE' AS crypto_symbol,
    r.date,
    r.open,
    r.high, 
    r.low,
    r.close,
    r.volume,
    r.adjclose,
    d.daily_return
FROM RAW_CRYPTO.DOGE_RAW_DAILY r
LEFT JOIN RAW_CRYPTO.DOGE_DAILY_RETURNS d ON r.date = d.date
WHERE r.date >= '2020-01-01';

In [ ]:
-- Create stream to track changes in the harmonized table
CREATE OR REPLACE STREAM HARMONIZED_CRYPTO.CRYPTO_HARMONIZED_STREAM 
ON TABLE HARMONIZED_CRYPTO.CRYPTO_HARMONIZED
SHOW_INITIAL_ROWS = TRUE;

In [ ]:
CREATE OR REPLACE FUNCTION HARMONIZED_CRYPTO.CALCULATE_VOLATILITY_UDF(prices ARRAY)
RETURNS FLOAT
LANGUAGE PYTHON
RUNTIME_VERSION = '3.8'
HANDLER = 'calculate_volatility'
PACKAGES = ('numpy')
AS
$$
import numpy as np

def calculate_volatility(prices):
    if prices is None or len(prices) < 2:
        return None
    
    # Calculate daily returns
    prices_array = np.array(prices)
    returns = np.diff(prices_array) / prices_array[:-1]
    
    # Calculate standard deviation of returns (volatility)
    volatility = np.std(returns) * 100  # Convert to percentage
    
    return float(volatility)
$$;